In [2]:
# 3.1.


from numpy import array
from numpy.linalg import solve, inv, cholesky, norm
from scipy.linalg import lu, solve_triangular
from typing import Tuple, Callable
from time import time


def solve1(A: array, f: array) -> array:
    return solve(A, f)


def solve2(A: array, f: array) -> array:
    return inv(A) @ f


def solve3(A: array, f: array) -> array:
    P, L, U = lu(A)
    f_perm = P @ f
    l = solve_triangular(L, f_perm, lower=True)
    u = solve_triangular(U, l)
    
    return u


def solve4(A: array, f: array) -> array:
    L = cholesky(A)
    l = solve_triangular(L, f, lower=True)
    l_herm = solve_triangular(L.conj().T, l)
    
    return l_herm


def measure_runtime(solve: Callable[[array, array], array],
                    A: array, f: array) -> Tuple[array, float]:
    start = time()
    x = solve(A, f)
    finish = time()
    
    return x, finish - start


def err_norm(x: array, y: array) -> float:
    return norm(x - y) / norm(x)


def res_norm(f: array, Ax: array) -> float:
    return norm(f - Ax) / norm(f)


def matrix_norm(A: array) -> float:
    return norm(A, 2) * norm(inv(A), 2)


def print_stats(solve: Callable[[array, array], array],
                A: array, y: array) -> None:
    f = A @ y
    x, runtime = measure_runtime(solve, A, f)
    print(f"Runtime: {runtime}")
    print(f"Error norm: {err_norm(x, y)}")
    print(f"Residuum norm: {res_norm(f, A @ x)}")
    if A.shape[0] <= 100 and A.shape[1] <= 100:
        print(f"Matrix A norm: {matrix_norm(A)}")


def compare_solvers(A: array, y: array) -> None:
    print_stats(solve1, A, y)
    print_stats(solve2, A, y)
    print_stats(solve3, A, y)
    print_stats(solve4, A, y)


In [ ]:
# 3.2.


from numpy.random import rand


def gen_input(n: int) -> Tuple[array, array]:
    A = rand(n, n)
    x = rand(n)
    
    return A @ A.T, x